In [1]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime
from crawlers.finlab.data_process_tools import last_month,year_transfer
from bs4 import BeautifulSoup

In [2]:
class CrawlInsiderHoldTW:
    def __init__(self,date):
        self.date=date
        self.target_name = "董事、監察人、經理人及大股東持股餘額彙總表"
        self.sub_market = ["sii", "otc", "rotc"]



    def crawl_main(self):
        url_date=last_month(self.date)
        url_month=lambda m:'0'+str(m) if m<10 else str(m)
        data = []
        for market in self.sub_market:
            url='https://siis.twse.com.tw/publish/'+market+'/'+str(url_date.year - 1911)+'IRB110_'+url_month(url_date.month)+'.HTM'
            headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko)'
                              ' Chrome/39.0.2171.95 Safari/537.36'}
            try:            
                r = requests.get(url, headers=headers)
                r.encoding = 'big5'
                df = pd.read_html(r.text)[0].iloc[2:,:]
                data.append(df)
            except ValueError :
                print(print(f'market:{market}**WARRN: Pandas cannot find any table in the HTML file'))
                pass
        try:
            df = pd.concat(data)
        except ValueError :
            return None
        df=df.astype(str)
        df.columns=['stock_name','issued_num','director_add','director_lower','director_hold',
                   'director_hold_ratio','manager_hold','big10_hold']
        df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda s: pd.to_numeric(s, errors="coerce"))
        df['stock_id']=df['stock_name'].apply(lambda s:s[:4])
        df['stock_name']=df['stock_name'].apply(lambda s:s[4:])
        df['date'] = datetime.date(self.date.year, self.date.month, 20)
        return df

In [3]:
C=CrawlInsiderHoldTW(datetime.datetime(2020,5,1))
C.crawl_main()

,stock_name,issued_num,director_add,director_lower,director_hold,director_hold_ratio,manager_hold,big10_hold,stock_id,date
2,臺灣水泥,5.66562e+09,0,0,7.85652e+08,13.86,5.30712e+06,0,1101,2020-05-20
3,亞洲水泥,3.36145e+09,0,0,9.75243e+08,29.01,2.7204e+06,7.50511e+08,1102,2020-05-20
4,嘉新水泥,7.74781e+08,0,0,1.3253e+08,17.1,87353,1.2737e+08,1103,2020-05-20
5,環球水泥,6.53609e+08,0,0,1.57717e+08,24.13,5.10402e+07,5.08883e+07,1104,2020-05-20
6,幸福水泥,4.04738e+08,0,0,1.60048e+07,3.95,1.4178e+07,5.2631e+07,1108,2020-05-20
...,...,...,...,...,...,...,...,...,...,...
246,台灣利得,4.5e+07,0,0,3.82879e+06,8.5,618371,0,8490,2020-05-20
247,真好玩娛,2.7456e+07,0,0,4.84108e+06,17.63,624710,0,8491,2020-05-20
248,兔將創意,2.15e+07,0,0,5.51412e+06,25.64,20000,3.49647e+06,8492,2020-05-20
249,經緯航太,3.4857e+07,0,0,7.46837e+06,21.42,110182,7.7347e+06,8495,2020-05-20


In [ ]:
from bs4 import BeautifulSoup


date=datetime.datetime(2020,4,1)
url_date=last_month(date)
url_month=lambda m:'0'+str(m) if m<8 else str(m)
market='rotc'
url='https://siis.twse.com.tw/publish/'+market+'/'+str(url_date.year - 1911)+'IRB110_'+url_month(url_date.month)+'.HTM'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko)'
                  ' Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get(url, headers=headers)
r.encoding = 'big5'
pd_html=pd.read_html(StringIO(r.text))
df = pd.DataFrame(pd_html[0]).iloc[2:,:]
df=df.astype(str)
df.columns=['stock_name','issued_num','director_add','director_lower','director_hold',
           'director_hold_ratio','manager_hold','big10hfold']
df['stock_id']=df['stock_name'].apply(lambda s:s[:4])
df['stock_name']=df['stock_name'].apply(lambda s:s[4:])
soup = BeautifulSoup(r.text)
data_date = soup.h3.text
data_date=data_date[-9:]
df.iloc[:, 2:-3] = df.iloc[:, 2:-3].apply(lambda s: pd.to_numeric(s, errors="coerce"))
df

In [ ]:
c=tag.text
data_date=c[-9:]
datetime.datetime.strptime(year_transfer(data_date),'%Y-%m-%d')
